In [1]:
import numpy as np
import pandas as pd
import re, sys, os, csv
from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

In [7]:
#stop_words = list(get_stop_words('en'))         #About 900 stop words
#nltk_words = list(stopwords.words('english'))   #About 150 stop words
#stop_words.extend(nltk_words)

def word_prob(word): return dictionary[word] / total
def words(text): return re.findall('[a-z]+', text.lower())
dictionary = Counter(words(open('dataset/wordlists/merged.txt').read()))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def fix_hashtag(text):
    text = text.group().split(":")[0]
    text = text[1:] # remove '#'
    try:
        test = int(text[0])
        text = text[1:]
    except:
        pass
    output = ' '.join(viterbi_segment(text)[0])
    #print(output)
    return output

def clean_tweet( tweet):
        tweet = tweet.lower()
        tweet = re.sub("(#[A-Za-z0-9]+)", fix_hashtag, tweet)
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def remove_stopwords(word_list):
        filtered_tweet=""
        for word in word_list:
            word = word.lower() 
            if word not in stopwords.words("english"):
                filtered_tweet=filtered_tweet + " " + word
        
        return filtered_tweet.lstrip()
    
def vectorise_label(label):
    if label == "empty":return 1 # neutral
    elif label == "sadness":return 2 # sad
    elif label == "enthusiasm":return 3 # happy
    elif label == "neutral":return 0 # neutral
    elif label == "worry":return 4 # sad
    elif label == "surprise":return 5 # happy
    elif label == "love":return 6 # happy
    elif label == "fun":return 7 # happy
    elif label == "hate":return 8
    elif label == "happiness":return 9 # happy
    elif label == "boredom":return 10 # neutral
    elif label == "relief":return 11 # happy
    elif label == "anger":return 12

In [8]:
data_train = pd.read_csv('dataset/data/text_emotion.csv', sep=',')
print("Dataset shape:",data_train.shape)
print(data_train.sentiment[0],":",data_train.content[0])

Dataset shape: (40000, 4)
empty : @tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[


In [9]:
dataWriter = csv.writer(open('data.csv', 'w'), delimiter=',',lineterminator="\n")

total = 40000
for i in range(40000):
    #print("Progress: ",round(i/total*100,2),"   ",end="\r")
    tweet= clean_tweet(data_train.content[i])
    #tweet = remove_stopwords(tweet.split())
    dataWriter.writerow([tweet, str(vectorise_label(data_train.sentiment[i]))])
    #sys.stdout.write("\033[F")
    
print("Progress: ",100,"\nComplete!")

Progress:  100 
Complete!


In [10]:
count = 0
with open('data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        count+=1

In [11]:
count

40000